In [1]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate
import miou_eval as mi

In [2]:
def combine_dims(a, start=0, count=2):
    """ Reshapes numpy array a by combining count dimensions, 
        starting at dimension index start """
    s = a.shape
    return np.reshape(a, s[:start] + (-1,) + s[start+count:])


def collate_fn(batch):
    return tuple(zip(*batch))

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

cuda


In [4]:
class MMCellDataset(Dataset):
    def __init__(self,root_dir, tester = False, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.img = []
        self.nml = []
        self.tester = tester
        self.datano =  root_dir.split('/')[1]
        print(self.datano)
        for im_name in os.listdir(self.root_dir):
            self.nml.append(im_name)
            # tmplist = []
            # lent = len(im_name[:-4])
            # print(im_name,im_name[:lent])
            # imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
            # tmplist.append(imgtmp[0])
            # tmplist.append(imgtmp[1])
            # tmplist.append(imgtmp[2])
            # for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            #     if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
            #         mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
            #         tmplist.append(mask)
            # tmplist = np.array(tmplist)
            # #print(tmplist.shape)
            # self.img.append(tmplist)   
    
    def __len__(self):
        return len([name for name in os.listdir(self.root_dir)])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        im_name = self.nml[idx]
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
        tmplist.append(imgtmp[0])
        tmplist.append(imgtmp[1])
        tmplist.append(imgtmp[2])
        for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
                tmplist.append(mask)
        tmplist = np.array(tmplist)
        #print("Tmplist: ",tmplist.shape)
        patch_stack = tmplist
        patch_stack = np.array(patch_stack)
        # k_w = random.randint(700,patch_stack.shape[2])
        # k_h = random.randint(700,patch_stack.shape[1])
        k_w = patch_stack.shape[2]
        k_h = patch_stack.shape[1]
        o_w = random.randint(0,patch_stack.shape[2]-k_w)
        o_h = random.randint(0,patch_stack.shape[1]-k_h)
        patch_img = patch_stack[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
        image_hsv = cv2.cvtColor(patch_img.transpose(1,2,0), cv2.COLOR_BGR2HSV ).transpose(2,0,1)
        image_lab = cv2.cvtColor(patch_img.transpose(1,2,0), cv2.COLOR_BGR2LAB ).transpose(2,0,1)
        #print(image_hsv.shape,image_lab.shape)
        patch_img = np.concatenate((patch_img,image_hsv,image_lab),0)
        #print("Patch img",patch_img.shape)
        instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = instances.transpose(1,2,0)
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(dispim/255)
            t=1
        if t==0:
            #print("Abort")
            if self.tester:
                return "Problem","Hao gai"
            else:
                #print("Abort Abort ")
                return self.__getitem__((idx+1)%len(self.nml))
        masks = np.array(masks)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)
        img = torch.as_tensor(patch_img, dtype=torch.float32)
        img = img/255
        data["boxes"] =  boxes
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        data["iscrowd"] = iscrowd
        data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        data["masks"] = masks
        data["area"] = area
        data["image_id"] = torch.tensor(idx)
        return img,data
train = MMCellDataset("../dataset4/images/")
test = MMCellDataset("../dataset4/test/")
fintest = MMCellDataset("../test/images/",True)
#train = torch.utils.data.ConcatDataset([train, test])
#print(len(test))
trainloader = DataLoader(train, batch_size=2, shuffle=True,collate_fn = collate_fn,num_workers=10)
testloader = DataLoader(test, batch_size=1, shuffle=True,collate_fn = collate_fn,num_workers=10)
fintestloader = DataLoader(fintest, batch_size=1, shuffle=True,collate_fn = collate_fn,num_workers=10)

dataset4
dataset4
test


In [5]:
class alchemy(nn.Module):
    def __init__(self,**kwargs) -> None:
        super(alchemy, self).__init__()
        self.vis = False
        self.channel_trans = nn.Sequential(
            nn.Conv2d(9,3,(1,1),stride=1,padding='same',bias=False),
            nn.BatchNorm2d(3,eps=1e-05, affine=True,),
            nn.Sigmoid()
        )
        self.maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,trainable_backbone_layers=5,**kwargs)
        in_features = self.maskrcnn.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
        self.maskrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one

    def forward(self, images, targets=None):
        for i in range(0,len(images)):
            images[i] = self.channel_trans(torch.stack([images[i]]))[0]
        ret = self.maskrcnn(images,targets)
        return ret

In [6]:
def evalm(model,fintestloader):
    model.eval()
    iou = 0
    cnt = 0
    for i, data in enumerate(fintestloader, 0):
        images, targets = data
        #print(type(targets))
        if(images[0]=="Problem"):
            continue
        #print(type(images[0]))
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        #print("Txnt: ",tcnt)
        del images,targets
        iou += tiou
        cnt += tcnt
        #print(iou/cnt)
    print("mIoU on test is:",iou.item()/cnt)
    model.train()
    return iou.item()/cnt
    

In [7]:
model = alchemy()
model.to(device)
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,weight_decay=0.00004)
model.train()
lmbda = lambda epoch: 0.8
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

In [9]:
min_val=0
for epoch in range(10): 
    model.train()
    lrs = 0
    for i, data in enumerate(trainloader, 0):
        lrs+=1
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        del images,targets
        #print("lr: ",optimizer.param_groups[0]['lr'])
        print(i,'loss:', losses.item())
        
        if lrs==200:
            lrs=0
            #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 200)
            # for param_group in optimizer.param_groups:
            #     param_group['lr'] = lr*0.6
            #     lr = lr * 0.6
            val = evalm(model,fintestloader)
            if(val>min_val):
                print("saved ",str(i)," with mIoU:",val)
                min_val=val
                torch.save(model.state_dict(), "alchemy_"+str(epoch)+"_"+str(val)[0:6]+".torch")

    torch.save(model.state_dict(), "alchemy_"+str(epoch)+".torch")

0 loss: 0.5115060210227966
1 loss: 0.4337990880012512
2 loss: 0.4162675738334656
3 loss: 0.3885021209716797
4 loss: 0.5949540734291077
5 loss: 0.5308842062950134
6 loss: 0.5620524287223816
7 loss: 0.3141358494758606
8 loss: 0.5415260791778564
9 loss: 0.2852074205875397
10 loss: 0.3456813395023346
11 loss: 0.5064498782157898
12 loss: 0.2972787618637085
13 loss: 0.5406436324119568
14 loss: 0.9897568821907043
15 loss: 0.9044831991195679
16 loss: 0.5168424844741821
17 loss: 0.33258917927742004
18 loss: 0.4518181085586548
19 loss: 2.5946929454803467
20 loss: 0.3890443742275238
21 loss: 0.3881169259548187
22 loss: 0.48951566219329834
23 loss: 0.6281388401985168
24 loss: 0.5180346369743347
25 loss: 0.6619378924369812
26 loss: 0.5675976276397705
27 loss: 0.7498211860656738
28 loss: 0.4330933690071106
29 loss: 0.46351516246795654
30 loss: 0.44123148918151855
31 loss: 0.28480714559555054
32 loss: 0.6842221021652222
33 loss: 0.5416366457939148
34 loss: 0.5565845370292664
35 loss: 0.48377880454063

KeyboardInterrupt: 

In [ ]:
for epoch in range(10): 


    # if epoch%1==0:
    #         model.eval()
    #         iou = 0
    #         cnt = 0
    #         for i, data in enumerate(fintestloader, 0):
    #             images, targets = data
    #             #print(type(targets))
    #             if(images[0]=="Problem"):
    #                 continue
    #             #print(type(images[0]))
    #             images = list(image.to(device) for image in images)
    #             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #             #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
    #             tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
    #             #print("Txnt: ",tcnt)
    #             del images,targets
    #             iou += tiou
    #             cnt += tcnt
    #             #print(iou/cnt)
    #         print("mIoU on test is:",iou.item()/cnt)

    model.train()


    for i, data in enumerate(trainloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(targets[0]["boxes"].shape)
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        del images,targets
        print(i,'loss:', losses.item())
    
    torch.save(model.state_dict(), "alchemy1_"+str(epoch)+".torch")
    if epoch%2==1:
        scheduler.step()


    model.eval()
    #evaluate(model, testloader, device=device)
    iou = 0
    cnt = 0
    
    for i, data in enumerate(testloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(type(model(images)[0]["masks"]))
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        del images,targets
        iou += tiou
        cnt += tcnt
    print("mIoU on val is:",iou.item()/cnt)

    if epoch%1==0:
            model.eval()
            iou = 0
            cnt = 0
            for i, data in enumerate(fintestloader, 0):
                images, targets = data
                #print(type(targets))
                if(images[0]=="Problem"):
                    continue
                #print(type(images[0]))
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
                tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
                #print("Txnt: ",tcnt)
                del images,targets
                iou += tiou
                cnt += tcnt
                #print(iou/cnt)
            print("mIoU on test is:",iou.item()/cnt)


print('Finished Training')

In [ ]:
del model
model = alchemy()
model.to(device)
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5)
model.train()
lmbda = lambda epoch: 0.5
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

In [ ]:
for epoch in range(3): 


    # if epoch%1==0:
    #         model.eval()
    #         iou = 0
    #         cnt = 0
    #         for i, data in enumerate(fintestloader, 0):
    #             images, targets = data
    #             #print(type(targets))
    #             if(images[0]=="Problem"):
    #                 continue
    #             #print(type(images[0]))
    #             images = list(image.to(device) for image in images)
    #             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #             #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
    #             tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
    #             #print("Txnt: ",tcnt)
    #             del images,targets
    #             iou += tiou
    #             cnt += tcnt
    #             #print(iou/cnt)
    #         print("mIoU on test is:",iou.item()/cnt)

    model.train()


    for i, data in enumerate(trainloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(targets[0]["boxes"].shape)
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        del images,targets
        print(i,'loss:', losses.item())
    
    torch.save(model.state_dict(), "alchemy2_"+str(epoch)+".torch")
    if epoch%2==1:
        scheduler.step()


    model.eval()
    #evaluate(model, testloader, device=device)
    iou = 0
    cnt = 0
    
    for i, data in enumerate(testloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(type(model(images)[0]["masks"]))
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        del images,targets
        iou += tiou
        cnt += tcnt
    print("mIoU on val is:",iou.item()/cnt)

    if epoch%1==0:
            model.eval()
            iou = 0
            cnt = 0
            for i, data in enumerate(fintestloader, 0):
                images, targets = data
                #print(type(targets))
                if(images[0]=="Problem"):
                    continue
                #print(type(images[0]))
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
                tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
                #print("Txnt: ",tcnt)
                del images,targets
                iou += tiou
                cnt += tcnt
                #print(iou/cnt)
            print("mIoU on test is:",iou.item()/cnt)


print('Finished Training')

In [ ]:
del model
model = alchemy()
model.to(device)
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
optimizer1 = torch.optim.AdamW(model.maskrcnn.parameters(), lr=8e-5)
optimizer2 = torch.optim.SGD(model.channel_trans.parameters(),lr=1e-3)
model.train()
lmbda = lambda epoch: 0.5
scheduler1 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer1, lr_lambda=lmbda)
lmbda1 = lambda epoch: 0.5
scheduler2 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer2, lr_lambda=lmbda1)

In [ ]:
for epoch in range(3): 


    # if epoch%1==0:
    #         model.eval()
    #         iou = 0
    #         cnt = 0
    #         for i, data in enumerate(fintestloader, 0):
    #             images, targets = data
    #             #print(type(targets))
    #             if(images[0]=="Problem"):
    #                 continue
    #             #print(type(images[0]))
    #             images = list(image.to(device) for image in images)
    #             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #             #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
    #             tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
    #             #print("Txnt: ",tcnt)
    #             del images,targets
    #             iou += tiou
    #             cnt += tcnt
    #             #print(iou/cnt)
    #         print("mIoU on test is:",iou.item()/cnt)

    model.train()


    for i, data in enumerate(trainloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(targets[0]["boxes"].shape)
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer1.step()
        optimizer2.step()
        del images,targets
        print(i,'loss:', losses.item())
    
    torch.save(model.state_dict(), "alchemy3_"+str(epoch)+".torch")
    # if epoch%2==1:
    #     scheduler1.step()

    scheduler2.step()
    model.eval()
    #evaluate(model, testloader, device=device)
    iou = 0
    cnt = 0
    
    for i, data in enumerate(testloader, 0):
        images, targets = data
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print(type(model(images)[0]["masks"]))
        tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
        del images,targets
        iou += tiou
        cnt += tcnt
    print("mIoU on val is:",iou.item()/cnt)

    if epoch%1==0:
            model.eval()
            iou = 0
            cnt = 0
            for i, data in enumerate(fintestloader, 0):
                images, targets = data
                #print(type(targets))
                if(images[0]=="Problem"):
                    continue
                #print(type(images[0]))
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
                tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
                #print("Txnt: ",tcnt)
                del images,targets
                iou += tiou
                cnt += tcnt
                #print(iou/cnt)
            print("mIoU on test is:",iou.item()/cnt)


print('Finished Training')

In [ ]:
model.eval()
iou = 0
cnt = 0
for i, data in enumerate(fintestloader, 0):
    images, targets = data
    #print(type(targets))
    if(images[0]=="Problem"):
        continue
    #print(type(images[0]))
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #print(model(images)[0]["masks"].squeeze().shape,targets[0]["masks"].shape)
    tiou,tcnt = mi.miou_eval(targets[0]["masks"],(model(images)[0]["masks"].squeeze(1)>0.5))
    #print("Txnt: ",tcnt)
    del images,targets
    iou += tiou
    cnt += tcnt
#print(iou/cnt)
print("mIoU on test is:",iou.item()/cnt)

In [ ]:
# model1 = alchemy(box_nms_thresh=0.5)
# model1.to(device)
# model1.load_state_dict(torch.load("alchemy_1_9128.torch"))
# model1.eval()
model1 = model
for im_name in os.listdir("./submission/x/"):
    imgtmp = cv2.imread("./submission/x/"+im_name,1).transpose(2,0,1)
    imgtmp = torch.as_tensor(imgtmp, dtype=torch.float32)
    imgtmp = imgtmp/255
    imgtmp = imgtmp.to(device)
    output = model1([imgtmp])
    prefix = im_name[0:-4]
    ref = 0
    for i in range(output[0]['masks'].shape[0]):
        print(output[0]['scores'][i].item())
        it = torch.clone(output[0]['masks'][i])
        it = it.detach().to('cpu').numpy()
        it = it.squeeze(0)
        it = (it>=0.5).astype(float)
        it = it*255
        cv2.imwrite("./submission/y/"+prefix+"_"+str(i)+".bmp",it)

In [ ]:
! make sub